In [1]:
import articles.article_fetch as article_fetch
import articles.articles_info as article_info
from articles import article

import clinical_trials.clinical_trial_fetch as ct_fetch
import clinical_trials.clinical_trials_info as ct_info
from clinical_trials import clinical_trial

import csv
import csv_data.csv as csv_data

from dataframe import create_dataframe
from dataframe import calculate_attributes

from common_functions import common_functions

import pandas as pd

import importlib

# I get the gold standard
gold_standard = pd.read_csv('ClinicalPmidsALL.csv', encoding = 'ISO-8859-1', sep = ';')

# Now I change the common answer from string to numerical
csv_data.numerical_answers(gold_standard)

# Let's correct data on csv (common answer sometimes is not correct)
csv_data.correctData(gold_standard)

# I get all the clinical trials ID and I get the articles in xml
Clinical_trials = ct_fetch.get_xml_doms(gold_standard['CT'].tolist())

# I get all the PMID and I get the articles in xml
PubMed_id_string = list(map(str, gold_standard['PMID'].tolist())) # I get the PMID as list of strings
PubMed_articles = article_fetch.fetch_many_articles(PubMed_id_string, local=True) # It takes time to fetch the articles

# Let's map every clinical trial to every article (and remove the one that are not present) and common answer
df = create_dataframe.get_base_dataframe(gold_standard, Clinical_trials, PubMed_articles)

# standard, spacy or stanford
ct_org_sample = 'standard'
ar_org_sample = 'spacy'

article 27953647  not found.
article 27948541  not found.
article 27943881  not found.
article 27949797  not found.
article 27955116  not found.
article 27950623  not found.
article 27945102  not found.
clinical trial NCT02659670  not found.
article 27198327  not found.


In [2]:
#lam = gold_standard['CT']
#lam.index[lam == 'NCT00003204'].tolist()[0]

ct_last_names, ct_first_name_initials, ct_first_names = ct_info.get_all_name_parts(df['CT'].tolist(), gold_standard)

df['ct_last_name'], df['ct_first_name_initial'], df['ct_first_name'] = [ct_last_names, ct_first_name_initials, ct_first_names]

In [3]:
# print(sum(x is not None for x in ct_last_names))
df = df.dropna().reset_index(drop = True)

In [4]:
ar_last_names, ar_first_name_initials, ar_first_names = article_info.get_all_name_parts(df['PubMed'].tolist(), gold_standard)

df['ar_last_name'], df['ar_first_name_initial'], df['ar_first_name'] = [ar_last_names, ar_first_name_initials, ar_first_names]
df.dropna().reset_index(drop = True)
print()

In [5]:
# now that we have the base information, let's add other attributes

# we get the organization of the principal investigator, so we don't need to pass his/her name
ct_organization_names = ct_info.get_all_organization_names(df['CT'].tolist(), df['ct_last_name'].tolist(),
                                                           df['ct_first_name_initial'].tolist(),
                                                           sample=ct_org_sample)

# Now we insert it in the dataframe
if('ct_organization' not in df):
    df.insert(6, 'ct_organization', ct_organization_names)
    

# I get the e-mails
ct_mails = ct_info.get_all_mails(df['CT'].tolist(), df['ct_last_name'].tolist(), df['ct_first_name_initial'].tolist())

if('ct_mail' not in df):
    df.insert(7, 'ct_mail', ct_mails)
    
# I get the year
ct_years = ct_info.get_all_years(df['CT'].tolist())
    
if('ct_year' not in df):
    df.insert(8, 'ct_year', ct_years)
    
# I get the initials of the name
ct_initials = ct_info.get_all_initials(df['ct_first_name'].tolist())

if('ct_initials' not in df):
    df.insert(9, 'ct_initials', ct_initials)
    
# I get the title
ct_titles = ct_info.get_all_titles(df['CT'].tolist())

if('ct_title' not in df):
    df.insert(10, 'ct_title', ct_titles)
    
# I get the country and the city
ct_countries, ct_cities = ct_info.get_all_countries_and_cities(df['CT'].tolist())

if('ct_country' not in df):
    df.insert(11, 'ct_country', ct_countries)
    
if('ct_city' not in df):
    df.insert(12, 'ct_city', ct_cities)

NCT00730210 doesn't have an organization.
NCT02220283 doesn't have an organization.


In [6]:
print(sum(x is not None for x in df['ct_mail']))

267


In [7]:
# Let's add the same attributes for the articles

ar_organization_names, ar_locations = article_info.get_all_organizations_locations(df['PubMed'].tolist(),
                                                           df['ar_last_name'].tolist(),
                                                           df['ar_first_name_initial'].tolist(),
                                                           sample = ar_org_sample)
df['ar_organization'] = ar_organization_names

ar_mails = article_info.get_all_mails(df['PubMed'].tolist(), df['ar_last_name'].tolist(),
                                      df['ar_first_name_initial'].tolist())
df['ar_mail'] = ar_mails

ar_years = article_info.get_all_years(df['PubMed'].tolist())

df['ar_year'] = ar_years

ar_initials = article_info.get_all_initials(df['ar_first_name'])

df['ar_initials'] = ar_initials

ar_titles = article_info.get_all_titles(df['PubMed'].tolist())

df['ar_title'] = ar_titles
df['ar_location'] = ar_locations

print(sum(x is not None for x in ar_organization_names))
print(sum(x is not None for x in ar_mails))

475
126


In [27]:
# I now calculate useful attributes for the classifiers
first_name_equalities = calculate_attributes.get_string_arrays_similarity(df['ct_first_name'].tolist(),
                                                                          df['ar_first_name'].tolist())
organization_similarities, organization_type_equalities = calculate_attributes.get_organization_similarity_and_type_equality(df['ct_organization'].tolist(),
                                                                             df['ar_organization'].tolist(),
                                                                             ct_org_sample, ar_org_sample)
email_equalities = calculate_attributes.get_arrays_equality(df['ct_mail'].tolist(), df['ar_mail'].tolist())
year_differences = calculate_attributes.get_year_differences(df['ct_year'].tolist(), df['ar_year'].tolist())
last_name_lengths = calculate_attributes.get_last_name_lengths(df['ct_last_name'].tolist())
initials_equality = calculate_attributes.get_arrays_equality(df['ct_initials'].tolist(), df['ar_initials'].tolist())
namespace_sizes = calculate_attributes.get_namespace_ambiguities(df['ct_last_name'].tolist(),
                                                                 df['ct_first_name_initial'].tolist())
country_equalities, city_equalities = calculate_attributes.get_location_equalities(df['ct_country'].tolist(),
                                                                                   df['ct_city'].tolist(),
                                                                                   df['ar_location'].tolist())
jds_similarities, sts_similarities = calculate_attributes.get_all_jds_sts_similarities(df['CT'].tolist(),
                                                                    df['PubMed'].tolist(), 4, 52,
                                                                    'percentage_ranking')

# Let's add the attributes to the data frame
df['first_name_equality'], df['organization_similarity'] = [first_name_equalities, organization_similarities]
df['email_equality'], df['year_difference'] = [email_equalities, year_differences]
df['last_name_length'], df['initials_equality'] = [last_name_lengths, initials_equality]
df['namespace_size'], df ['country_equality'] = [namespace_sizes, country_equalities]
df['city_equality'], df['organization_type_equality'] = [city_equalities, organization_type_equalities]
df['jds_similarity'], df['sts_similarity'] = [jds_similarities, sts_similarities]

In [28]:
# Now let's normalize and/or standardize the attributes that need this
df['namespace_size'] = common_functions.normalize(df['namespace_size'])
df['last_name_length'] = common_functions.normalize(df['last_name_length'])
df['year_difference'] = common_functions.normalize(df['year_difference'])

'''
import numpy as np

np_array = np.array(df['namespace_size'].tolist())
std = np_array.std()
mean = np_array.mean()
for i in range(len(np_array)):
    np_array[i] = (np_array[i] - mean)/std
    
df['namespace_size'] = np_array.tolist()

np_array = np.array(df['last_name_length'].tolist())
std = np_array.std()
mean = np_array.mean()
for i in range(len(np_array)):
    np_array[i] = (np_array[i] - mean)/std
    
df['last_name_length'] = np_array.tolist()

np_array = np.array(df['year_difference'].tolist())
std = np_array.std()
mean = np_array.mean()
for i in range(len(np_array)):
    np_array[i] = (np_array[i] - mean)/std
    
df['year_difference'] = np_array.tolist()
'''

"\nimport numpy as np\n\nnp_array = np.array(df['namespace_size'].tolist())\nstd = np_array.std()\nmean = np_array.mean()\nfor i in range(len(np_array)):\n    np_array[i] = (np_array[i] - mean)/std\n    \ndf['namespace_size'] = np_array.tolist()\n\nnp_array = np.array(df['last_name_length'].tolist())\nstd = np_array.std()\nmean = np_array.mean()\nfor i in range(len(np_array)):\n    np_array[i] = (np_array[i] - mean)/std\n    \ndf['last_name_length'] = np_array.tolist()\n\nnp_array = np.array(df['year_difference'].tolist())\nstd = np_array.std()\nmean = np_array.mean()\nfor i in range(len(np_array)):\n    np_array[i] = (np_array[i] - mean)/std\n    \ndf['year_difference'] = np_array.tolist()\n"

In [29]:
# Now we delete the ones we don't need in the classifier
df_to_save = df.drop(['CT', 'PubMed', 'ct_last_name', 'ct_first_name_initial', 'ct_first_name',
              'ct_organization', 'ct_mail', 'ct_year', 'ct_initials', 'ct_title', 'ct_country', 'ct_city', 
                'ar_last_name','ar_first_name_initial','ar_first_name','ar_organization', 'ar_mail', 'ar_initials',
                      'ar_year', 'ar_title', 'ar_location'], axis = 1)

In [30]:
# Writing the dataframe to file
import csv
import codecs
csv = df_to_save.to_csv(index=False)

file = codecs.open("dataframe.csv", "w", "utf-8")
file.write(csv)
file.close()

In [31]:
importlib.reload(article_info)
importlib.reload(ct_info)
importlib.reload(calculate_attributes)
importlib.reload(clinical_trial)
importlib.reload(common_functions)

<module 'common_functions.common_functions' from 'C:\\Users\\Brescia\\Anaconda3\\Diploma\\common_functions\\common_functions.py'>

In [32]:
df.iloc[40]

CT                            <clinical_trials.clinical_trial.ClinicalTrial ...
PubMed                        <articles.article.Article object at 0x0000014F...
common_answer                                                                 0
ct_last_name                                                               saba
ct_first_name_initial                                                         s
ct_first_name                                                           samir f
ct_organization                                        University of Pittsburgh
ct_mail                                                                    None
ct_year                                                                    2005
ct_initials                                                                  sf
ct_title                          Genetic Predictors of Outcome in HCM Patients
ct_country                                                      [United States]
ct_city                                 

In [33]:
df

,CT,PubMed,common_answer,ct_last_name,ct_first_name_initial,ct_first_name,ct_organization,ct_mail,ct_year,ct_initials,...,email_equality,year_difference,last_name_length,initials_equality,namespace_size,country_equality,city_equality,organization_type_equality,jds_similarity,sts_similarity
0,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,abdulkarim,b,bassam,AHS Cancer Control Alberta,None,2008,b,...,0,0.000000,0.470588,1,0.000627,0,0,0,0.629170,0.441395
1,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,gawin,f,frank,"Friends Research Institute, Inc.",None,1999,f,...,0,0.045455,0.176471,1,0.000534,0,0,0,0.217699,0.103129
2,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,deutsch,s,steven,Washington D.C. Veterans Affairs Medical Center,None,1999,s,...,0,0.181818,0.294118,1,0.005065,0,0,0,0.000000,0.034479
3,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,maisiak,r,richard s,University of Alabama at Birmingham,None,1999,rs,...,0,0.106061,0.294118,1,0.000421,0,0,0,0.032278,0.028387
4,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,gorden,p,phillip,National Institute of Diabetes and Digestive a...,gordenp@extra.niddk.nih.gov,1999,p,...,0,0.196970,0.235294,1,0.003333,1,1,1,0.661631,0.418099
5,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,hochster,h,howard,Eastern Cooperative Oncology Group,None,2000,h,...,0,0.121212,0.352941,1,0.001638,1,0,0,0.267082,0.374291
6,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,fabian,c,carol j,University of Kansas Medical Center,bkimler@kumc.edu,2000,cj,...,0,0.106061,0.235294,0,0.002163,0,0,0,0.423678,0.374529
7,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,dematteo,r,ronald,American College of Surgeons,None,2001,r,...,0,0.227273,0.352941,1,0.004512,0,0,0,0.140057,0.345120
8,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,0,klein,j,jonathan d,University of Rochester,None,2002,jd,...,0,0.181818,0.176471,0,0.042269,1,0,1,0.000000,0.035419
9,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,dispenzieri,a,angela,Mayo Clinic,None,2002,a,...,0,0.015152,0.529412,1,0.006422,0,0,0,0.545796,0.270666


In [15]:
lista = df['CT'].tolist()
a = df['ct_last_name'].tolist()
b = df['ct_first_name_initial'].tolist()
mails = df['ct_mail'].tolist()

count = 0

for i in range(len(lista)):
    tags = lista[i].get_all_name_tags(a[i],b[i])
    
    for tag in tags:
        if tag.email is not None or mails[i]:
            #print(tag.email, mails[i])
            count += 1
        
print(count)

501


In [16]:
lista = df['CT'].tolist()

for i in range(len(lista)):
    loc = lista[i].clinical_trial.findAll('location')
    if len(loc) > 1:
        print(loc[0].country.text, loc[1].country.text, '\n')

United States United States 

United States United States 

United States United States 

United States United States 

United States United States 

United States United States 

United States United States 

Hong Kong Hong Kong 

United States United States 

United States United States 

United States United States 

Japan Japan 

United States United States 

Canada Canada 

United States United States 

United States United States 

United States United States 

United States United States 

Canada Canada 

Japan Japan 

United States United States 

United States Korea, Republic of 

United States United States 

United States United States 

United States United States 

Denmark Denmark 

United States United States 

United Kingdom United Kingdom 

United States United States 

Belgium Belgium 

United States United States 

Canada Canada 

United States United States 

United States United States 

United States United States 

United States United States 

United States Unite

In [17]:
lista = df['CT'].tolist()

for i in range(len(lista)):
    loc = lista[i].clinical_trial.findAll('overall_official')
    if len(loc) >= 1:
        print(loc[0], '\n')

<overall_official>
<last_name>Bassam Abdulkarim, MD, FRCPC</last_name>
<role>Principal Investigator</role>
<affiliation>AHS Cancer Control Alberta</affiliation>
</overall_official> 

<overall_official>
<last_name>Frank Gawin, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Friends Research Institute, Inc.</affiliation>
</overall_official> 

<overall_official>
<last_name>Steven Deutsch, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Washington D.C. Veterans Affairs Medical Center</affiliation>
</overall_official> 

<overall_official>
<last_name>Richard S. Maisiak, PhD, MSPH</last_name>
<role>Principal Investigator</role>
<affiliation>University of Alabama at Birmingham</affiliation>
</overall_official> 

<overall_official>
<last_name>Phillip Gorden, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK)</affiliation>
</overall_official> 

<overall_official>
<last_name

</overall_official> 

<overall_official>
<last_name>Nikhil Munshi, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Dana-Farber Cancer Institute</affiliation>
</overall_official> 

<overall_official>
<last_name>Joseph R Leventhal, MD, PhD</last_name>
<role>Principal Investigator</role>
<affiliation>Northwestern University</affiliation>
</overall_official> 

<overall_official>
<last_name>Tyng-Guey Wang, MD</last_name>
<role>Principal Investigator</role>
<affiliation>National Taiwan University, School of Medicine, National Taiwan University</affiliation>
</overall_official> 

<overall_official>
<last_name>John Bantle, MD</last_name>
<role>Principal Investigator</role>
<affiliation>University of Minnesota - Clinical and Translational Science Institute</affiliation>
</overall_official> 

<overall_official>
<last_name>Pei-Fang Tang, PhD</last_name>
<role>Principal Investigator</role>
<affiliation>National Taiwan University Hospital</affiliation>
</overall_official> 

<overall

</overall_official> 

<overall_official>
<last_name>Stephen J Brown, MD</last_name>
<role>Principal Investigator</role>
<affiliation>AIDS Research Alliance of West Hollywood</affiliation>
</overall_official> 

<overall_official>
<last_name>Diana M Antoniucci, MD</last_name>
<role>Principal Investigator</role>
<affiliation>University of California, San Francisco</affiliation>
</overall_official> 

<overall_official>
<last_name>Diana M Antoniucci, MD</last_name>
<role>Principal Investigator</role>
<affiliation>University of California, San Francisco</affiliation>
</overall_official> 

<overall_official>
<last_name>Marc J Semigran, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Massachusetts General Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>Steven J. Lichtenstein, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Investigator / Contact</affiliation>
</overall_official> 

<overall_official>
<last_name>Richard Aplenc</last_na

</overall_official> 

<overall_official>
<last_name>John Wagner, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Thomas Jefferson University</affiliation>
</overall_official> 

<overall_official>
<last_name>Michael W. Otto, Ph.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Boston University</affiliation>
</overall_official> 

<overall_official>
<last_name>Philip Gilbert, FDSRCS FRCS</last_name>
<role>Principal Investigator</role>
<affiliation>Queen Victoria Hospital, East Grinstead</affiliation>
</overall_official> 

<overall_official>
<last_name>Michelle Keightley</last_name>
<role>Principal Investigator</role>
<affiliation>University of Toronto</affiliation>
</overall_official> 

<overall_official>
<last_name>Berthold Amann, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Franziskus Hospital, Berlin Vascular Center</affiliation>
</overall_official> 

<overall_official>
<last_name>CEM Griffiths, Professor</last_name>
<role>Principal Inv

</overall_official> 

<overall_official>
<last_name>Thomas Bandholm, MSc</last_name>
<role>Principal Investigator</role>
<affiliation>Gait Analysis Laboratory, Department of Orthopedic Surgery, Hvidovre University Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>Danielle Andrade, MD</last_name>
<role>Principal Investigator</role>
<affiliation>College of Physicians and Surgeons of Ontario</affiliation>
</overall_official> 

<overall_official>
<last_name>Dan V Iosifescu, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Massachusetts General Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>Wing King Lee, Dr</last_name>
<role>Principal Investigator</role>
<affiliation>Yaumatei Psychiatric Center, Kwai Chung Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>Yixing Jiang, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Penn State College of Medicine</affiliation>
</overall_official> 

...

</overall_official> 

<overall_official>
<last_name>Peter A LeWitt, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Henry Ford Hospital - West Bloomfield</affiliation>
</overall_official> 

<overall_official>
<last_name>Vladimir M Kushnir, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Washington University School of Medicine</affiliation>
</overall_official> 

<overall_official>
<last_name>Tudor C Poerner, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Universitätsklinikum Jena</affiliation>
</overall_official> 

<overall_official>
<last_name>Ronen Rubinshtein, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Carmel Medical Center</affiliation>
</overall_official> 

<overall_official>
<last_name>Raymond Cross, MD, MS</last_name>
<role>Principal Investigator</role>
<affiliation>University of Maryland</affiliation>
</overall_official> 

<overall_official>
<last_name>José Valber Meneses, MD</last_name>
<role>Study Chair</ro

</overall_official> 

<overall_official>
<last_name>Linda Fleisher, PhD, MPH</last_name>
<role>Principal Investigator</role>
<affiliation>Children's Hospital of Philadelphia</affiliation>
</overall_official> 

<overall_official>
<last_name>Glen E Foster, Ph.D.</last_name>
<role>Principal Investigator</role>
<affiliation>University of British Columbia</affiliation>
</overall_official> 

<overall_official>
<last_name>Meera Shah, MBChB</last_name>
<role>Principal Investigator</role>
<affiliation>Mayo Clinic</affiliation>
</overall_official> 

<overall_official>
<last_name>Jun Jun She, M.D.; PhD.</last_name>
<role>Principal Investigator</role>
<affiliation>First Affiliated Hospital Xi'an Jiaotong University</affiliation>
</overall_official> 

<overall_official>
<last_name>Roberto Bottino, Doctor</last_name>
<role>Principal Investigator</role>
<affiliation>Fondazione Poliambulanza Istituto Ospedaliero</affiliation>
</overall_official> 

<overall_official>
<last_name>Richard Laing, MD</last_

</overall_official> 

<overall_official>
<last_name>Athena Zuppa, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Children's Hospital of Philadelphia</affiliation>
</overall_official> 

<overall_official>
<last_name>Gavin Murphy</last_name>
<role>Principal Investigator</role>
<affiliation>University of Leicester</affiliation>
</overall_official> 

<overall_official>
<last_name>Petr Neuzil, Prof.</last_name>
<role>Principal Investigator</role>
<affiliation>Na Homolce Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>Bas Kietselaer, M.D. PhD</last_name>
<role>Principal Investigator</role>
<affiliation>Maastricht UMC</affiliation>
</overall_official> 

<overall_official>
<last_name>Yongjun Fang, Ph.D</last_name>
<role>Principal Investigator</role>
<affiliation>Nanjing Children's Hospital</affiliation>
</overall_official> 

<overall_official>
<last_name>lixin xie, Ph.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Chinese PLA General

</overall_official> 

<overall_official>
<last_name>Seong Woo Jeon, Professor</last_name>
<role>Principal Investigator</role>
<affiliation>Gastric Cancer Center, Kyungpook National University Medical Center</affiliation>
</overall_official> 

<overall_official>
<last_name>Deanna L Gibson, PhD</last_name>
<role>Principal Investigator</role>
<affiliation>University of British Columbia - Okanagan</affiliation>
</overall_official> 

<overall_official>
<last_name>AYong Yu, MD. PhD.</last_name>
<role>Principal Investigator</role>
<affiliation>Wenzhou Medical University</affiliation>
</overall_official> 

<overall_official>
<last_name>Ernest Seidman, MD</last_name>
<role>Principal Investigator</role>
<affiliation>Research Institute of the MUHC</affiliation>
</overall_official> 

<overall_official>
<last_name>Xichun Hu, MD, PhD</last_name>
<role>Principal Investigator</role>
<affiliation>Fudan University</affiliation>
</overall_official> 

<overall_official>
<last_name>Jeffrey Krischer, Ph.D.<

</overall_official> 

<overall_official>
<last_name>Daniel J Cox, PhD</last_name>
<role>Principal Investigator</role>
<affiliation>University of Virginia</affiliation>
</overall_official> 

<overall_official>
<last_name>Mikhail Efanov, MD, PhD</last_name>
<role>Study Chair</role>
<affiliation>Moscow Clinical Scientific Center</affiliation>
</overall_official> 

<overall_official>
<last_name>Sebastien Levesque, MD</last_name>
<role>Principal Investigator</role>
<affiliation>University Hospital of Poitiers, Department of Cardiology</affiliation>
</overall_official> 

<overall_official>
<last_name>Eman Omran, M.D.</last_name>
<role>Principal Investigator</role>
<affiliation>Cairo University</affiliation>
</overall_official> 

<overall_official>
<last_name>Thorsten Rudroff, Ph.D.</last_name>
<role>Principal Investigator</role>
<affiliation>University of Colorado, Denver</affiliation>
</overall_official> 

<overall_official>
<last_name>Roger Bedimo, MD</last_name>
<role>Principal Investigat

In [18]:
lista = df['organization_similarity'].tolist()
for i in range(len(lista)):
    print(lista[i], df['ct_organization'][i], '\n',df['ar_organization'][i], '\n\n')

0.0 AHS Cancer Control Alberta 
 None 


0.0 Friends Research Institute, Inc. 
 None 


0.0 Washington D.C. Veterans Affairs Medical Center 
 None 


0.0 University of Alabama at Birmingham 
 None 


0.6111111111111112 National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) 
 ['Obesity Branch', 'National Institute of Diabetes and Digestive', 'National Institutes of Health'] 


0.32352941176470584 Eastern Cooperative Oncology Group 
 ['Division of Medical Oncology', 'New York University School of Medicine', 'NYU Cancer Institute'] 


0.0 University of Kansas Medical Center 
 None 


0.37777777777777777 American College of Surgeons 
 ['Columbia University College of Physicians and', 'Weill Cornell Medical College'] 


0.5 University of Rochester 
 ['University of California, Irvine', 'CA'] 


0.0 Mayo Clinic 
 None 


0.0 NCI, NIH 
 None 


0.0 Joslin Diabetes Center 
 None 


0.0 Children's Oncology Group 
 None 


0.0 University of Washington 
 None 


0.0 University a

 ['Nuclear Physics and Technology', 'Peking University'] 


0.045454545454545414 Université de Montréal 
 ['Gulf Medical University', 'UAE'] 


0.0 Dept. of Psychiatry, China Medical University Hospital, Taichung, Taiwan 
 None 


0.44897959183673475 Aarhus University, Perinatal Epidemiological Unit 
 ['Perinatal Epidemiology Research Unit', 'Aarhus University Hospital'] 


0.0 Yamaguchi University Hospital 
 None 


0.0 University of California San Francisco Medical Center-Mount Zion 
 None 


0.9080459770114943 Eunice Kennedy Shriver National Institute of Child Health and Human Development (NICHD) 
 ['Perinatology Research Branch', 'Program for Perinatal Research and Obstetrics, Division of Intramural Research', 'Eunice Kennedy Shriver National Institute of Child Health and Human Development', 'National Institutes of Health', 'MI', 'Department of Obstetrics and Gynecology', 'University of Michigan', 'MI', 'Department of Epidemiology and Biostatistics', 'Michigan State University', 'M

 ['the Department of Neurology', 'University of California', 'Department of Neurology', 'Neurological Institute', 'University Hospitals Case Medical Center', 'Case Western Reserve University'] 


0.0 State University of New York - Upstate Medical University 
 None 


0.5 Baylor College of Medicine 
 ['Department of Pediatrics, Baylor College of Medicine', 'Lille Frank Abercrombie Section of Cardiology', "Texas Children's Hospital"] 


0.0 Innovaderm Research Inc. 
 None 


0.0 Duke University 
 None 


0.0 Department of Internal Medicine 
 None 


0.0 CLINICAL TESTING CENTER of BEVERLY HILLS 
 None 


0.6511627906976745 George Washington University 
 ['Pediatrics', 'Division of Newborn Service', 'George Washington University Medical Center', "Children's National Medical Center"] 


0.0 University of Washington 
 None 


0.0 St. Jude Children's Research Hospital 
 None 


0.1785714285714286 Hospital Clinic of Barcelona 
 ['Institute of Liver Studies', "King's College Hospital", "King's 

 ['Department of Gynecology', 'Charité - University Medicine Berlin'] 


0.0 National Bureau of Economic Research 
 None 


0.21212121212121215 New England School of Acupuncture 
 ['Laboratory of Behavioral Genetics', 'Brain Mind Institute', 'Ecole Polytechnique Federale de Lausanne'] 


0.31999999999999995 University of Nottingham 
 ['Department of Psychology, University of Pittsburgh', 'PA', 'PA'] 


0.0 Cork Universtiy Hospital 
 None 


0.0 University of Crete 
 None 


0.0 Radboud University 
 None 


0.0 Ophthalmic Consultants of Long Island 
 [] 


0.0 National Institute of Mental Health (NIMH) 
 None 


0.0 University Hospital Zurich, Ophtalmic Clinic 
 None 


0.0 Mayo Clinic 
 None 


0.0 Centre François Baclesse 
 None 


0.0 University of Michigan 
 None 


0.0 University of Calgary 
 None 


0.1515151515151515 Hospital Italiano de Buenos Aires 
 ['Department of Health Informatics'] 


0.0 University of Californa, San Francisco 
 None 


0.0 Sinai Hospital of Baltimore 
 No

0.0 McGill University Health Center 
 None 


0.48484848484848486 Ascensia Diabetes Care 
 ['the Wellcome Trust-Medical Research Council Institute of Metabolic Science', 'University of Cambridge', 'H.R.M.', 'Wolfson Diabetes', 'Cambridge University Hospitals NHS Foundation Trust', 'H.R.M.', 'the Elsie Bertram Diabetes Centre', 'R.C.T.', 'H.R.M.', 'the Department of Obstetrics and Gynaecology', 'K.P.S.', 'Norwich University Hospitals NHS Foundation Trust', 'the Norwich Medical School', 'University of East Anglia', 'H.R.M.', 'Ipswich Hospital NHS Trust', 'the Division of Epidemiology and Biostatistics, Leeds Institute of Cardiovascular', 'Metabolic Medicine', 'University of Leeds, Leeds', 'G.R.L.'] 


0.0 Glostrup University Hospital, Copenhagen 
 None 


0.19047619047619047 Exempla Saint Joseph Hospital 
 ['Department of Biological Sciences', 'School of Pharmacy and Biochemistry, University of Buenos Aires', 'IQUIFIB-CONICET'] 


0.0 Imperial College London 
 None 


0.5 Martin-Luther-U

1.0 Massachusetts General Hospital 
 ['Diabetes Unit', 'Department of Medicine', 'Massachusetts General Hospital', 'Harvard Medical School'] 


0.15254237288135597 Université de Montréal 
 ['Department of Obstetrics and Gynecology , Angers University'] 


0.0 University of Colorado, Denver 
 None 


0.15000000000000002 Beijing No.6 Hospital 
 ['Department of Psychiatry, Columbia University', 'The New York State Psychiatric Institute'] 


0.0 IQ healthcare, Radboud University Nijmegen Medical Centre, Nijmegen, the Netherlands 
 None 


0.0 Basel Institute for Clinical Epidemiology and Biostatistics, University Hospital Basel 
 None 


0.16666666666666663 Massachusetts General Hospital 
 ['Department of Surgery', 'Department of Medicine', 'Department of Radiation Oncology', '4 Department of Radiology, Pancreatic Cancer Program', 'The Medical College of Wisconsin'] 


0.25 Johns Hopkins University/Sidney Kimmel Cancer Center 
 ['Department of Pharmaceutical Sciences', 'Hampton University'

 ['Department of Neuroscience', 'Norwegian University of Science and Technology', 'NTNU', 'Department of Neurosurgery', 'St. Olavs University Hospital', 'National Advisory Unit', 'Spinal Surgery Center', 'St. Olavs University Hospital', 'Norwegian Centre of Competence', 'St. Olavs University Hospital'] 


0.0 McGill University Health Center 
 None 


0.0 Division of Vascular Surgery, UNC at Chapel Hill 
 None 


0.5 Peking University 
 ['Department of Pharmacy', 'National Taiwan University Hospital', 'Graduate Institute of Clinical Pharmacy', 'National Taiwan University'] 


0.7692307692307692 Rockefeller Univesrity 
 ['The Rockefeller University'] 


0.0 Children's Hospital Medical Center, Cincinnati 
 None 


0.0 Memorial Sloan Kettering Cancer Center 
 None 


0.0 Chinese Academy of Medical Sciences 
 None 


0.0 Medical Research Council 
 None 


0.0 Taichung Veteran General Hospital, Taiwan 
 None 


1.0 University of Utah 
 ['Department of Pharmaceutics and Pharmaceutical Chemist

0.19999999999999996 Nationwide Children's Hospital 
 ['Michigan State University', 'Kalamazoo Center for Medical Studies', 'Oakland Drive'] 


0.2954545454545454 Emergency Medicine Physicians 
 ['University of Nevada School of Medicine', 'Department of Emergency Medicine', 'University Medical Center of Southern Nevada', 'Department of Emergency Medicine'] 


0.0 University of Cagliari 
 None 


0.16326530612244894 Associate Professor 
 ['Department of Epidemiology, University of Florida'] 


0.0 Frederiksberg University Hospital 
 None 


0.3548387096774194 University of Michigan 
 ['Division of Nuclear Medicine', 'Department of Radiology, University of Michigan Medical School'] 


0.0 -Hôpital Robert Debré, Service de psychiatrie de l'enfant et de l'adolescent 
 None 


0.0 Illinois Retina and Eye Associates 
 None 


0.5833333333333333 Loma Linda University 
 ['Department of Biology', 'Johns Hopkins University'] 


1.0 New York Institute of Technology 
 ['Department of Osteopathic Me

 None 


0.6363636363636364 The Royal Wolverhampton NHS Trust 
 ['Deansley Centre', 'Royal Wolverhampton Hospital'] 


0.0 University of Virginia 
 None 


0.2093023255813954 Shanghai Chest Hospital 
 ['Department of Medicine', 'Johns Hopkins University School of Medicine'] 


0.0 OHSU Knight Cancer Institute 
 None 


0.0 University of California, San Diego 
 None 


0.19999999999999996 Azienda Ospedaliera Universitaria di Bologna Policlinico S. Orsola Malpighi 
 ['Division of Neonatology, Department of Pediatrics'] 


0.0 Hôpital Hôtel-Dieu 
 None 


0.18181818181818177 Pope Research Corp. 
 ['Division of Rheumatology, Department of Medicine', 'The University of Western Ontario'] 


0.13793103448275867 Paracelsus Medical University 
 ['Klinik'] 


0.0 The University of Arizona Cancer Center at Dignity Health St. Joseph's Hospital and Medical Center 
 None 


0.0 Beijing 302 Hospital 
 None 


0.42000000000000004 Cleveland Clinic, Case Comprehensive Cancer Center 
 ['Cleveland Clinic 

In [19]:
lista = df['CT'].tolist()
abstracts = []

count = 0
for i in range(len(lista)):
    if lista[i].clinical_trial.eligibility.criteria is not None:
        count +=1
        #print(lista[i].clinical_trial.find('nct_id'))
        print(lista[i].clinical_trial.eligibility.criteria.textblock.text)
        
print(count)


        Inclusion Criteria:

          -  newly diagnosed GBM

          -  Diagnosis must be established by open biopsy or tumour resection

          -  Tumour must have a supratentorial component

          -  Over 18 years

          -  pre-treatment evaluations must be met

          -  study therapy to begin within 6 weeks of surgery

          -  KPS greater or equal to 70

          -  patients must sign informed consent

          -  If female, patients must not be pregnant or lactating

          -  Women of childbearing potential and male participants must practice adequate
             contraception

        Exclusion Criteria:

          -  prior invasive malignancy )except for non-melanomatous skin cancer) unless disease
             free for greater than 3 years

          -  recurrent or multifocal malignant gliomas

          -  metastatic disease of leptomeningeal spread

          -  prior chemo or radiosensitizers for cancers of the head and neck region

          

      

        Inclusion Criteria:

          -  Patient must have an ECOG/Zubrod performance status of ≤ 2

          -  Patient must have a diagnosis of high-risk primary GIST; NOTE: High risk is defined as
             tumor size ≥ 10 cm in maximum dimension, or the presence of tumor rupture before or
             during surgery, intraperitoneal hemorrhage or multifocal (< 5) intraperitoneal tumors

          -  Patient must have undergone complete gross resection (includes R0 [negative
             microscopic margins] and R1 [positive microscopic margins] resections) of a primary
             GIST within 70 days prior to registration

          -  Patient must have a histologic diagnosis of GIST that is confirmed by central
             pathology review

          -  Patient's tumor must stain positive for the Kit receptor tyrosine kinase on
             immunohistochemistry as determined by the central pathologist using the Dako (Dako
             Corp., Carpinteria, CA) anti-CD

      

        Inclusion Criteria:

          -  Diagnosed with type 1 diabetes mellitus within 3 months prior to study entry

          -  Positive for IAA, GAD65, or IA2 antibodies OR positive for GAD65 or IA2 antibodies
             after 2 weeks of starting insulin treatment

        Exclusion Criteria:

          -  History of treatment with any oral hypoglycemic agent for more than 3 months

          -  Ongoing use of medications known to influence glucose tolerance

          -  History of immunosuppressive or steroid therapy for more than 3 months within the 2
             years prior to study entry

          -  Severe active liver, heart, kidney, or immunodeficiency disease that may limit life
             expectancy or may require immunosuppression during the study

          -  Prior complications related to routine vaccinations

          -  Prior participation in a trial for prevention of type 1 diabetes mellitus. Individuals
             who are known to have been in t

      

        Eligibility:

          -  Aged 75 years or less AND 50% or more blockage of at least one artery in the heart,

          -  Heart problems in the last 3 years (including previous heart attack),

          -  Previous heart bypass surgery, or previous heart angioplasty with or without a stent,
             AND at least 6 natural teeth present in the mouth AND evidence of mild to severe gum
             problems.

        Exclusion:

        Any of the following:

          -  A major illness requiring hospitalization,

          -  Renal dialysis,

          -  Serum creatinine > 3 mg/dl,

          -  Organ transplant recipient requiring immunosuppression medication,

          -  Surgery needed in the next 3 years,

          -  Chemotherapy in the past 3 years,

          -  Head and/or neck radiation at any time in the past,

          -  Liver dysfunction,

          -  Class IV congestive heart failure,

          -  Drug or alcohol abuse,

          -  Pacemaker 

      

        Inclusion criteria

          -  HIV infection

          -  History of a CD4 cell count less than 200 cells/mm3

          -  Stable antiretroviral regimen

          -  English-speaking

        Exclusion criteria

          -  Inability or unwillingness to fill out questionnaires

          -  History of non-HIV related life-threatening disease
      

        Inclusion Criteria:

          -  Patients must have surgically incurable colon or rectal adenocarcinoma

          -  Karnofsky Performance Status of 60% or greater

          -  Patients may have symptomatic neuropathy

          -  Patients must have a life expectancy of at least 4 months

          -  Patients must be alimenting, receiving at least 1500Kcal/day nutrition, by any route

          -  Patients must have bidimensional measurable disease for response assessment

          -  Patients may have received adjuvant chemotherapy with fluoropyrimidine therapy

          -  Patients must have recovered 

...

      

        Inclusion Criteria:

          -  Ages 18-35

          -  Elevated anxiety sensitivity (ASI-3 score >=23)

          -  Able to safely participate in moderate-intensity physical activity as assessed by the
             Physical Activity Readiness Questionnaire (PAR-Q), which will be administered during
             screening

        Exclusion Criteria:

          -  Current aerobic exercise routine - Measured using the L-CAT with an established cut
             point of >2 (assessed at screening)

          -  Receiving ongoing psychotherapy or counseling (Self-report at screening)

          -  Recent change in psychotropic medication for anxiety (Self-report at screening)

          -  Psychiatric hospitalization in past 6 months (Self-report at screening)

          -  Current pregnancy (Self-report at screening)
      

        Inclusion Criteria:

          1. surgical indication for lobectomy;

          2. minimal invasive surgery;

          3. ASA (American S

      

        Inclusion Criteria:

          -  Age between 18 and 85 years of age

          -  Participants must have undergone implantation of DBS electrodes.

          -  Participants must be able to understand the purpose of this research and must sign the
             informed consent form.

          -  Participants must understand that the role of this research is to enhance our
             understanding of brain functioning and that he/she will not directly or indirectly
             benefit from the study.

        Exclusion Criteria:

          -  Participants who have serious cognitive or psychological impairments and cannot give
             informed consent.

          -  Participants who are unable to effectively or efficiently communicate, for example
             patients suffering from speech deficits (dysarthria, aphasia) or are non-English
             speaking.
      

        Inclusion Criteria:

          1. Age 18-40 years

          2. Documented HIV-infect

      

        Inclusion Criteria:

          1. Age ≥40 (to reduce the likelihood of enrolling patients with obstruction due to
             primary sclerosing cholangitis)

          2. Cholestatic liver function tests, including serum alkaline phosphatase level ≥ 300
             IU/L and bilirubin level ≥ 3.7 mg/dL

          3. Radiographic evidence of a biliary hilar stricture OR intrahepatic but no extrahepatic
             biliary ductal dilation

        Exclusion Criteria:

          1. Known radiographic evidence of a Bismuth-Corlette type 1 biliary stricture

          2. Known diagnosis of primary sclerosing cholangitis without suspicion of dominant hilar
             stricture

          3. Recent gallbladder/biliary surgery within 12 months

          4. Known Mirizzi syndrome

          5. Known IgG4-mediated cholangiopathy

          6. Significant liver metastatic disease interfering with safe/effective PTBD

          7. Significant ascites interfering with safe/eff

      

        Inclusion Criteria:

          -  HIV-uninfected women, aged 15-24 years, being sexually active and having engaged in
             any form of transactional sex at least once in the last 3 months, agreeing to
             participate in all intervention sessions and to all study procedures and interviews
             planned over 18 months of follow-up.

        Exclusion Criteria:

          -  presence of any physical or mental condition likely to cause inability to provide
             consent or hinder study participation; HIV infection
      

        Inclusion Criteria:

          -  Patient is scheduled for a bronchoscopic lung volume treatment using one-way valves

          -  Patient has provided signed informed consent.

        Exclusion Criteria:

        For safety reasons patients that meet the following criteria will not be included in this
        study:

          1. FEV1 <20%

          2. RV/TLC>70%

          3. pCO2 >6.5

          4. RVSP>40mmHg



      

        -  INCLUSION CRITERIA:

          -  Ages between 18-70 years.

          -  Right-handed (tested by the Edinburg handedness inventory).

          -  Able to give informed consent and able to comply with all study procedures.

        EXCLUSION CRITERIA:

          -  Self-reported consumption of >14 alcoholic drinks/week for a man and >7 alcoholic
             drinks/week for a woman.

          -  Abnormal findings on neurological examination.

          -  History of brain tumor, stroke, head trauma with loss of consciousness, epilepsy or
             seizures.

          -  Current episode of any major psychiatric illness.

          -  Use of medications that act directly on the CNS.

          -  Hearing loss reported in the history or detected in the routine physical examination

          -  Having permanent tattooed makeup (eyeliner, lip, etc.) or general tattoos. Subjects
             with tattoos will be excluded if those are in a dangerous location in the b

In [20]:
lista = df['PubMed'].tolist()
abstracts = []

count = 0
for i in range(len(lista)):
    coso = lista[i].article.findAll('MeshHeading')
    if len(coso) == 2:
        print(coso[0].text)
        
print(count)


Emergency Medicine
trends


Famous Persons


Humans

0


In [21]:
last_names = df['ct_last_name'].tolist()
first_initial = df['ct_first_name_initial'].tolist()

names_array = []
for i in range(len(last_names)):
    string = last_names[i] + '+' + first_initial[i] + '[author]'
    names_array.append(string)

In [22]:
lista = df['PubMed'].tolist()
roba = []

for i in range(len(lista)):
    med = lista[i].article.MedlineJournalInfo.Country
    roba.append(med.text)
    
coso = set(roba)
print(coso)

{'England', 'Czech Republic', 'Indonesia', 'Denmark', 'Poland', 'Brazil', 'New Zealand', 'Slovakia', 'Uganda', 'Spain', 'Austria', 'Lebanon', 'France', 'Canada', 'Argentina', 'Korea (South)', 'Saudi Arabia', 'South Africa', 'Turkey', 'Pakistan', 'China', 'Iran', 'Sweden', 'Greece', 'Japan', 'United States', 'Ireland', 'India', 'Italy', 'Netherlands', 'Costa Rica', 'Singapore', 'Switzerland', 'Germany', 'Thailand', 'Ukraine', 'Bulgaria', 'Australia', 'United Arab Emirates'}


In [23]:
lista = df['CT'].tolist()
roba = []
count = 0

for i in range(len(lista)):
    country = lista[i].clinical_trial.find('location')
    if country is None:
        count += 1
        continue
    roba.append(country.country.text)
    
coso = set(roba)
print(coso)
print(count)

{'Egypt', 'Jordan', 'Denmark', 'Poland', 'Serbia', 'Indonesia', 'Belgium', 'Portugal', 'Colombia', 'Brazil', 'Czechia', 'Vietnam', 'United Kingdom', 'Spain', 'Lithuania', 'Iran, Islamic Republic of', 'Uganda', 'Burkina Faso', 'Austria', 'Zambia', 'Lebanon', 'Mexico', 'Zimbabwe', 'Iraq', 'Hong Kong', 'France', 'Canada', 'Argentina', 'Korea, Republic of', 'Kuwait', 'Russian Federation', 'Saudi Arabia', 'Israel', 'Bangladesh', 'Slovenia', 'Turkey', 'Pakistan', 'Malawi', 'Hungary', 'Chile', 'Cameroon', 'China', 'Nigeria', 'Finland', 'Sweden', 'Japan', 'Greece', 'Burundi', 'Taiwan', 'United States', 'Ireland', 'India', 'Tanzania', 'Italy', 'Netherlands', 'Ghana', 'Singapore', 'Switzerland', 'Germany', 'Thailand', 'Ukraine', 'Croatia', 'Norway', 'Kenya', 'Australia', 'Mali'}
87


In [24]:
df

,CT,PubMed,common_answer,ct_last_name,ct_first_name_initial,ct_first_name,ct_organization,ct_mail,ct_year,ct_initials,...,email_equality,year_difference,last_name_length,initials_equality,namespace_size,country_equality,city_equality,organization_type_equality,jds_similarity,sts_similarity
0,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,abdulkarim,b,bassam,AHS Cancer Control Alberta,None,2008,b,...,0,0.000000,0.470588,1,0.000627,0,0,0,0.516345,0.415668
1,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,gawin,f,frank,"Friends Research Institute, Inc.",None,1999,f,...,0,0.045455,0.176471,1,0.000534,0,0,0,0.205678,0.099415
2,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,deutsch,s,steven,Washington D.C. Veterans Affairs Medical Center,None,1999,s,...,0,0.181818,0.294118,1,0.005065,0,0,0,0.011458,0.035696
3,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,maisiak,r,richard s,University of Alabama at Birmingham,None,1999,rs,...,0,0.106061,0.294118,1,0.000421,0,0,0,0.042765,0.032953
4,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,gorden,p,phillip,National Institute of Diabetes and Digestive a...,gordenp@extra.niddk.nih.gov,1999,p,...,0,0.196970,0.235294,1,0.003333,1,1,1,0.583667,0.398814
5,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,hochster,h,howard,Eastern Cooperative Oncology Group,None,2000,h,...,0,0.121212,0.352941,1,0.001638,1,0,0,0.242221,0.352993
6,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,fabian,c,carol j,University of Kansas Medical Center,bkimler@kumc.edu,2000,cj,...,0,0.106061,0.235294,0,0.002163,0,0,0,0.363632,0.353336
7,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,dematteo,r,ronald,American College of Surgeons,None,2001,r,...,0,0.227273,0.352941,1,0.004512,0,0,0,0.147187,0.324725
8,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,0,klein,j,jonathan d,University of Rochester,None,2002,jd,...,0,0.181818,0.176471,0,0.042269,1,0,1,0.043593,0.036725
9,<clinical_trials.clinical_trial.ClinicalTrial ...,<articles.article.Article object at 0x0000014F...,1,dispenzieri,a,angela,Mayo Clinic,None,2002,a,...,0,0.015152,0.529412,1,0.006422,0,0,0,0.469777,0.256853


In [25]:
lista = df['ct_country'].tolist()
lista1 = df['ct_city'].tolist()
lista2 = df['ar_location'].tolist()
lista3 = df['country_equality'].tolist()
lista4 = df['city_equality'].tolist()

for i in range(len(lista)):
    if(lista2[i] is not None):
        print(lista[i], lista1[i], lista2[i], lista3[i], lista4[i], '\n\n')

['United States'] ['Bethesda'] ['Bethesda', 'Maryland', 'USA'] 1 1 


['United States'] ['Boston'] ['USA'] 1 0 


['United States'] ['Durham'] ['Surgeons', 'New York', 'NY'] 0 0 


['United States'] ['Rochester'] ['USA'] 1 0 


['Japan'] ['Okayama'] ['Okayama', 'Japan'] 1 1 


['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States'] ['La Puente', 'Athol', 'Ann Arbor', 'Detroit', 'Brooklyn', 'New York', 'Montpelier'] ['Newton', 'MA', 'USA'] 1 0 


['United States'] ['Columbus'] ['Columbus'] 1 1 


['United States'] ['Houston'] ['Houston', 'Texas', 'USA'] 1 1 


['United States'] ['Rochester'] ['Philadelphia', 'PA', 'USA', 'Philadelphia', 'Philadelphia', 'PA', 'USA'] 1 0 


['Brazil'] ['Porto Alegre'] ['RS', 'Brazil'] 1 0 


['United States'] ['Boston'] ['Boston', 'MA', 'USA'] 1 1 


['United States'] ['New Brunswick'] ['Boston', 'Massachusetts', 'USA'] 1 0 


['Taiwan'] ['Taipei'] ['Taipei', 'Taiwan'] 1 1 


['United States'

['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States'] ['San Diego', 'Washington', 'Palm Beach', 'Tampa', 'Chicago', 'Lexington', 'Silver Spring', 'Pascagoula', 'Rochester', 'Cleveland', 'Lancaster', 'Knoxville', 'Holladay', 'Falls Church'] ['Salt Lake City', 'USA'] 1 0 


['Canada'] ['Hamilton'] ['Canada', 'Canada', 'Canada'] 1 0 


['Belgium', 'Belgium', 'Belgium'] ['Antwerpen', 'Edegem', 'Wilrijk'] ['Belgium'] 1 0 


['United States'] ['Milwaukee'] ['Milwaukee', 'Wis.'] 1 1 


['United States'] ['Atlanta'] ['Atlanta', 'Atlanta', 'Ga.'] 1 1 


['United States', 'United States', 'United States'] ['Boston', 'Ann Arbor', 'St. Louis'] ['Boston'] 1 1 


[] [] [] 0 0 


['United States'] ['Baltimore'] ['Baltimore', 'Maryland', 'USA'] 1 1 


['United States'] ['Baltimore'] ['Baltimore', 'Maryland', 'USA'] 1 1 

['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United State

['United States', 'United States'] ['Louisville', 'Louisville'] ['Louisville', 'KY'] 1 1 


['Taiwan'] ['Taipei'] ['China'] 0 0 


['Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy'] ['Alessandria', 'Ancona', 'Arezzo', 'Aviano', 'Brescia', 'Brescia', 'Brindisi', 'Caltanissetta', 'Carrara', 'Cecina', 'Cremona', 'Cuneo', 'Empoli', 'Fabriano', 'Firenze', 'Genova', 'Lecce', 'Legnano', 'Lido di Camaiore', 'Livorno', 'Lucca', 'Milano', 'Milano', 'Monza', 'Napoli', 'Novara', 'Parma', 'Perugia', 'Perugia', 'Pesaro', 'Pescia', 'Piombino', 'Pisa', 'Pisa', 'Pisa', 'Pistoia', 'Prato', 'Reggio Emilia', 'Rome', 'Rome', 'Rome', 'Siena', 'Sondrio', 'Torino'] ['It

['Australia', 'Germany', 'Germany', 'Germany', 'Sweden', 'Sweden', 'Sweden', 'Sweden', 'Sweden', 'Switzerland', 'Switzerland', 'Switzerland', 'Switzerland', 'Switzerland', 'Switzerland', 'Switzerland', 'Turkey', 'Turkey', 'Turkey', 'Turkey'] ['Epping', 'Berlin', 'Marburg', 'München', 'Falun', 'Lund', 'Stockholm', 'Stockholm', 'Uppsala', 'Basel', 'Bern', 'Bern', 'Genève', 'Lausanne', 'Lugano', 'Zürich', 'Adana', 'Istanbul', 'Istanbul', 'Istanbul'] [] 0 0 


['Denmark'] ['Hvidovre'] ['Denmark'] 1 0 


['United States'] ['Durham'] ['Durham', 'NC', 'USA'] 1 1 


[] [] ['London', 'London', 'United Kingdom'] 0 0 


['United States'] ['San Francisco'] ['San Francisco General Hospital', 'San Francisco', 'USA'] 1 1 


['United States'] ['Ann Arbor'] ['Ann Arbor', 'Michigan'] 1 1 


['United States'] ['San Francisco'] ['San Francisco', 'San Francisco', 'Peter.Rechmann@ucsf.edu'] 1 1 


['United States'] ['Baltimore'] [] 0 0 


['United States'] ['Houston'] ['Houston', 'USA'] 1 1 


['United Stat

['United States', 'Denmark'] ['Greenville', 'Copenhagen'] ['Denmark'] 1 0 


['Lithuania'] ['Kaunas'] ['Kaunas', 'Lithuania'] 1 1 


['United States'] ['Bethesda'] ['Bethesda', 'MD', 'USA'] 1 1 


['Japan'] ['Komagane'] ['Komagane', 'Matsumoto', 'Japan'] 1 1 


['United States'] ['Lyons'] ['Lyons', 'New Jersey', 'USA'] 1 1 


['Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada'] ['Calgary', 'Nanaimo', 'Vancouver', 'Saint John', "St. John's", 'Dartmouth', 'Sudbury', 'Toronto'] [] 0 0 


['China'] ['Guangzhou'] ['Guangzhou', 'China'] 1 1 


['United States'] ['Baltimore'] ['Baltimore', 'Maryland', 'USA'] 1 1 


['United States'] ['Mishawaka'] ['M.E.W.', 'R.H.', 'S.H.', 'D.S.', 'Cambridge', 'Norfolk', 'E.M.S.', 'the United Kingdom'] 0 0 


['United States'] ['Denver'] ['Argentina', 'Argentina'] 0 0 


[] [] ['Halle', 'Germany', 'Addis Ababa', 'Ethiopia', 'Boston', 'Massachusetts', 'USA'] 0 0 


['Iran, Islamic Republic of'] ['Tehran'] ['Tehran', 'Iran'] 1 1 


[

['United States'] ['Birmingham'] ['Indianapolis', 'USA'] 1 0 


['Taiwan'] ['Taipei'] ['Taipei', 'Taiwan'] 1 1 


['China'] ['Nanning'] ['Nanning', 'China'] 1 1 


['United Kingdom', 'United Kingdom', 'United Kingdom', 'United Kingdom'] ['Bournemouth', 'Cardiff', 'Oswestry', 'Southampton'] ['Australia'] 0 0 


['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States'] ['Mesa', 'Phoenix', 'Carlsbad', 'Castro Valley', 'Fresno', 'Laguna Hills', 'Los Angeles', 'San Francisco', 'Sylmar', 'Hialeah', 'Miami', 'South Miami', 'Chicago', 'Oak Park', 'Indianapolis', 'Baltimore', 'Boston

['China', 'Germany', 'Switzerland'] ['Fuzhou', 'Herzogenaurach', 'Rheinfelden'] ['Fujian', 'China'] 1 0 


['France'] ['Caen'] ['Caen', 'France'] 1 1 


['Korea, Republic of'] ['Gangnam-Gu'] ['Bochum', 'Germany'] 0 0 


['France'] ['Caen'] ['France'] 1 0 


['China'] ['Beijing'] ['Gainesville', 'USA'] 0 0 


[] [] ['Cairo', 'Egypt'] 0 0 


['United States'] ['Saint Louis'] ['Fujian', 'China'] 0 0 


['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States'] ['Irvine', 'San Diego', 'Jacksonville', 'Atlanta', 'Chicago', 'Charlestown', 'Saddle Brook', 'New York', 'White Plains', 'Cleveland', 'Charleston', 'Seattle'] ['USA'] 1 0 


['United States'] ['Nashville'] ['Nashville'] 1 1 


['United States', 'United States', 'United States', 'United States', 'United States'] ['Tempe', 'San Diego', 'Plano', 'San Antonio', 'North Chesterfield'] ['North Ch

['United States'] ['Tampa'] ['Tampa', 'FL', 'USA'] 1 1 


['Mexico'] ['Leon'] ['San Carlos la Roncha', 'Mexico'] 1 0 


['Korea, Republic of'] ['Seoul'] ['Beijing', 'China'] 0 0 


['Finland'] ['Lahti'] ['Finland'] 1 0 


['United States'] ['Madison'] [] 0 0 


['China'] ['Wuhan'] ['Wuhan', 'Hubei Province', 'China'] 1 1 


[] [] ['Rochester', 'USA'] 0 0 


['United States'] ['Old Westbury'] ['NY', 'USA'] 1 0 


[] [] [] 0 0 


['Italy'] ['Udine'] ['Italy'] 1 0 


['United States'] ['Rochester'] ['Toronto', 'ON', 'Canada'] 0 0 


['China'] ["Xi'an"] ["Xi'an", 'China'] 1 1 


['United States'] ['Bronx'] ['Bronx', 'NY', 'USA'] 1 1 


['United States', 'United States', 'United States'] ['Miami', 'Chicago', 'Burlington'] ['Burlington', 'Massachusetts'] 1 1 


['Taiwan'] ['Taipei'] ['China'] 0 0 


['United States'] ['Fresno'] ['07743 Jena', 'Germany'] 0 0 


['Switzerland'] ['Basel'] ['Basel', 'Switzerland'] 1 1 


['China'] ['Beijing'] ['Beijing', 'China'] 1 1 


['United States', 'United

In [26]:
lista = df['ct_organization'].tolist()
lista1 = df['ar_organization'].tolist()
lista2 = df['organization_type_equality'].tolist()

for i in range(len(lista)):
    if lista1[i] is not None:
        print(lista[i], lista1[i], lista2[i], '\n\n')

National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) ['Obesity Branch', 'National Institute of Diabetes and Digestive', 'National Institutes of Health'] 1 


Eastern Cooperative Oncology Group ['Division of Medical Oncology', 'New York University School of Medicine', 'NYU Cancer Institute'] 0 


American College of Surgeons ['Columbia University College of Physicians and', 'Weill Cornell Medical College'] 0 


University of Rochester ['University of California, Irvine', 'CA'] 1 


Dept. of Immunology, Okayama University Schhol of Medicine and Dentistry ['Faculty of Health and Welfare', 'Kawasaki University of Medical Welfare', 'Kurashiki'] 1 


Inflexxion, Inc. ['Pain and Opioid Division', 'Inflexxion Inc.'] 1 


Ohio State University ['Division of Hematology', 'The Ohio State University-James Comprehensive Cancer Center'] 1 


Michael E DeBakey VA Medical Center ['Health Services Research and Development Center of Excellence'] 0 


University of Rochester Sleep Res

McMaster University ['Department of Family Medicine', 'McMaster University', 'Kingston Community Health Centres', 'Department of Family Medicine', "Queen's University"] 1 


Universiteit Antwerpen ['University of Ghent'] 0 


Medical College of Wisconsin ['Division of Critical Care Medicine', 'Department of Pediatrics', "Children's Hospital of Wisconsin"] 0 


Children's Healthcare of Atlanta ['Sibley Heart Center Cardiology', "Children's Healthcare"] 0 


Dana-Farber Cancer Institute ['Department of Medical Oncology', 'Dana-Farber Cancer Institute', 'Harvard Medical School'] 1 


Mayo Graduate School of Medicine ['Gastroenterology & Hepatology'] 0 


Johns Hopkins University ['Johns Hopkins University School of Medicine'] 1 


Johns Hopkins University ['Johns Hopkins University School of Medicine'] 1 


Ohio State University ['Department of Neurology', 'Ohio State University'] 1 


M.D. Anderson Cancer Center ['The Center for Personalized Cancer Therapy', 'University of California San

Johns Hopkins University ['Stanford University Medical Center', 'University of California Los Angeles Medical Center', 'CA', 'Indiana University Medical Center', 'Weill Medical College of Cornell University', 'University of Nebraska Medical Center', 'NE', 'Johns Hopkins University Oncology Center', 'British Columbia Cancer Agency Centre for Lymphoid Cancer', 'Toronto-Sunnybrook Regional Cancer Centre', 'Cross Cancer Institute', 'Washington University School of Medicine', 'MO', 'Oregon Health Science University', 'Rush University Medical Center', 'IL', 'University of Iowa Hospitals and Clinics', 'IA', 'Mayo Clinic', 'Dana-Farber Cancer Institute', 'Rocky Mountain Cancer Centers'] 1 


Oregon Health and Science University ['Neurology and Research', 'Institute of Integrative Medicine', 'Chair for Theory of Medicine', 'Integrative', 'Anthroposophic Medicine', 'Witten/Herdecke University'] 1 


Synergy Health Solutions ['Research and Development Division of the National Association of Count

Dartmouth-Hitchcock Medical Center ['Division of Hematology', 'The Ohio State University-James Comprehensive Cancer Center'] 0 


Region Syddanmark ['Department of Neurology', 'University of California', 'CA'] 0 


Stanford University ['Department of Obstetrics and Gynecology', 'Stanford University School of Medicine'] 1 


Beijing Chao Yang Hospital ['Nuclear Physics and Technology', 'Peking University'] 0 


University Medical Center Groningen ['University of Groningen, Department of Pharmacy, Division of Pharmacoepidemiology and Pharmacoeconomics'] 1 


University of Jena ['Department of Nutritional Physiology, Institute of Nutrition', 'Friedrich Schiller University', 'Dornburger Straße'] 1 


Valley Retina Institute ['USDA', 'Bee Biology & Systematics Laboratory', 'Utah State University'] 0 


PharmaNet ['Université Laval', 'Association'] 0 


Universitaire Ziekenhuizen Leuven ['Department of Vascular and Endovascular Surgery', 'Department of Interventional Radiology', 'University 

Mayo Clinic ['Prosserman Centre for Health Research', 'Lunenfeld-Tanenbaum Research Institute', 'Mount Sinai Hospital'] 0 


First Affiliated Hospital Xi'an Jiaotong University ['Department of General Surgery', 'First Affiliated Hospital of Medical College', "Xi'an Jiao-Tong University"] 1 


Albert Einstein College of Medicine ['Department of Oncology', 'Montefiore Medical Center/Albert Einstein College of Medicine'] 0 


Lahey Hospital & Medical Center ['Department of Neurosurgery, Lahey Clinic Medical Center'] 0 


Tri-Service General Hospital ['Department of Central Laboratory', 'Xuan Wu Hospital of Capital Medical University'] 1 


University of California, San Francisco ['Institut', 'Abbe School of Photonics', 'Max-Wien'] 0 


University Hospital, Basel, Switzerland ['Department for Anesthesia, Surgical Intensive Care', 'Prehospital Emergency Medicine', 'Pain Therapy', 'University Hospital Basel'] 1 


Cancer Institute and Hospital, Chinese Academy of Medical Sciences ['Chinese A